In [ ]:
import gradio as gr
import requests
from transformers import pipeline
import os
from dotenv import load_dotenv # to load environment variables from conventionally .env named files  

# HF pipeline for instruction-following
load_dotenv()

# Optionally, set your OpenWeatherMap key or simulate
# OPENWEATHERMAP_API_KEY -- >  https://home.openweathermap.org/api_keys
WEATHER_API_KEY = os.environ["OPENWEATHERMAP_API_KEY"]  # Or simulate


OPENWEATHERMAP_API_KEY= "YOUR_API_KEY"

In [2]:
# from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# # Method 1: Using pipeline directly (simpler)
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")  
# # Example usage with pipeline
# inputs = tokenizer("You are an assistant respond politely: How are you?", return_tensors="pt")
# outputs = model.generate(**inputs)
# decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(decoded_output)

In [ ]:
pipe = pipeline("text2text-generation", model="google/flan-t5-base")

# Example usage with pipeline 
# this is useful for prompt engineering and model testing, before deploying into an app or a agentic ai as an orchestration layer or a tool 
input_text = "You are an assistant, respond politely and help me with the weather.?"
result = pipe(input_text)
print(result[0]['generated_text'])


Device set to use mps:0


No, I don't know.


In [ ]:


# Cell 2: Weather function
def get_weather(city):
    try:
        # Add error handling for the API request 
        # GET API call to OpenWeatherMap for a weather structured report -- >
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
        r = requests.get(url) # --> THE GET request to the OpenWeatherMap API
        #############  r = requests.get(url) #############  
        # Contains variables with useful information about the request
        # status_codes - > 200, 404, 500, etc. to handle errors and information flow 
        # Reports -> to save in a variable as object -> in data 
        r.raise_for_status()  # Raise an exception for bad status codes
        # IF successful, gets a JSON formatted weather report 
        # ELSE IF unsuccessful, raises an exception
        
        data = r.json()
        if 'weather' not in data or 'main' not in data:
            return f"Error: Invalid response format for {city}"
            
        desc = data['weather'][0]['description']
        temp = data['main']['temp']
        print(data)
        # return f"The weather in {city} is {desc} with a temperature of {temp}°C."
        return city,desc,temp # --> The returned strings from the json object, will be used to create a user defined-structured report
    # depending on the status code, the error message will be different and the handling will be different
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"Error parsing weather data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"



In [20]:
get_weather("London")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 17.52, 'feels_like': 16.55, 'temp_min': 16.86, 'temp_max': 18.34, 'pressure': 1016, 'humidity': 47, 'sea_level': 1016, 'grnd_level': 1012}, 'visibility': 10000, 'wind': {'speed': 4.63, 'deg': 40}, 'clouds': {'all': 53}, 'dt': 1747849532, 'sys': {'type': 2, 'id': 268730, 'country': 'GB', 'sunrise': 1747800024, 'sunset': 1747857246}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


('London', 'broken clouds', 17.52)

In [22]:
def get_suggestion(weather_desc, temp):
    """Helper function to generate weather-appropriate suggestions"""
    # suggestion list, which is populated with suggestions depending on the temperature  - A rule based suggestion system <  -  >
    suggestions = []
    
    # Temperature based suggestions
    if temp >= 20:
        suggestions.append("It's quite warm - don't forget sunscreen and stay hydrated!")
    elif temp <= 10:
        suggestions.append("It's cold - remember to dress warmly!")
        
    # Weather condition based suggestions -> we transform weather_desc to lower case and check for keywords
    if any(word in weather_desc.lower() for word in ['rain', 'shower', 'drizzle', 'thunderstorm']):
        # then we append in addition to the previous suggestion 
        suggestions.append("Take an umbrella with you!")
    elif any(word in weather_desc.lower() for word in ['clear', 'sunny', 'sun']):
        suggestions.append("Don't forget your sunglasses!")
    elif 'cloud' in weather_desc.lower():
        suggestions.append("It might be changeable weather - be prepared!")
        
        # returns a string with the suggestions not a list 
    return " ".join(suggestions) if suggestions else ""

In [7]:
# # Agent logic — simulate reasoning and reflection
# def agent_response(user_input):
#     # Simplified and more direct prompt
#     prompt = f"""Question: {user_input}
#     If this is a weather question, respond with 'WEATHER' followed by the city name.
#     If not, respond naturally to the question."""
    
#     result = qa_pipeline(prompt)[0]["generated_text"].strip()
    
#     if "weather" in user_input.lower():
#         try:
#             # Improved city extraction
#             city = user_input.lower().split("weather in")[1].strip().rstrip('?').capitalize()
#             city,desc,temp = get_weather(city)

#             return f"The weather in {city} is {desc} with a temperature of {temp}°C \n {get_suggestion(desc, temp)}"
#         except Exception as e:
#             return f"I couldn't get the weather information for that location. Error: {str(e)}"
#     else:
#         return result

In [23]:
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")


Device set to use mps:0


In [ ]:


def agent_response(user_input):
    # main function to handle user input 

    # Direct weather detection via simple pattern
    # if in the user_input, which we decapitalize and split, we find the string "weather in" then 
    if "weather in" in user_input.lower():
        try:
            # naive city name extraction

            # Basic city name extraction
            #1. user_input.lower() - Converts the input text to lowercase
            #2. split("weather in")[1] - Splits the text at "weather in" and takes everything after it
            #3. strip() - Removes leading and trailing whitespace
            #4. rstrip('?') - Removes question mark from the end if present
            #5. capitalize() - Capitalizes the first letter of the city name -- > this is necessary convention for the API weather call 
            city = user_input.lower().split("weather in")[1].strip().rstrip('?').capitalize()
            print(city)
            city, desc, temp = get_weather(city)
                # when we get the variables from the get_weather function, we construct a string with the weather information and also append the suggestion string
            return f"The weather in {city} is {desc} with a temperature of {temp}°C.\n\n{get_suggestion(desc, temp)}"
        except Exception as e:
            # if we get an error then we return a string with the error message e which is the excepetion message received from the get_weather function
            return f"I couldn't get the weather information for that location. Error: {str(e)}"
    
    # Fall back to model if user input isn't explicit -- ie weather in does not exist in the user_input
    # this worked in our tests  -
    # input_text = "You are an assistant, respond politely and help me with the weather.?"
    #
    prompt = (
        f"You are an assistant, respond politely and help me with the weather. {user_input}"
    )

    result = qa_pipeline(prompt)
    # result = qa_pipeline(prompt)[0]["generated_text"]
    print(result[0]['generated_text'])
    if "WEATHER" in result[0]['generated_text'].upper():
        try:
            # city = result[0]['generated_text'].split(":")[1].strip().capitalize()
            # city, desc, temp = get_weather(city)
            city = user_input.lower().split("weather")[1].strip().rstrip('?').capitalize()
            city, desc, temp = get_weather(city)

            return f"{result[0]['generated_text']}.\n\n The weather in {city} is {desc} with a temperature of {temp}°C.\n\n{get_suggestion(desc, temp)}"
        except Exception as e:
            return f"I couldn't get the weather information for that location. Error: {str(e)}"
    else:
        return result[0]['generated_text']    
    
    

In [27]:
agent_response("what are you")  # Example call to test the agent response

a teacher


'a teacher'